<a href="https://colab.research.google.com/github/ttogle918/news_by_kobert/blob/master/STS/BERTNextSentenceModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NextSentencePrediction

다음에 온 문장이 맞는지 예측


참고
+ [Bert 공식 Docs](https://huggingface.co/docs/transformers/model_doc/bert)
+ [한국어 언어모델 종류](https://littlefoxdiary.tistory.com/81)

## package 가져오기

In [ ]:
!pip install sentencepiece transformers torch

In [ ]:
!pip install seqeval
!pip install torchtext pytorch-lightning
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [ ]:
!pip install sentence_transformers numpy scikit-learn scipy nltk tqdm

In [1]:
import pytorch_lightning as pl
pl.__version__

'1.5.10'

In [2]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [3]:
from transformers import BertConfig, AdamW
from transformers import BertForNextSentencePrediction
from transformers import BertTokenizer

In [4]:
from tqdm import tqdm, tqdm_notebook
from typing import Callable, List, Tuple
from seqeval.metrics import accuracy_score

In [5]:
import torch
# gpu 연산이 가능하면 'cuda:0', 아니면 'cpu' 출력
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device, torch.cuda.device_count()

(device(type='cuda', index=0), 1)

## DataSet 가져오기

In [6]:
from google.colab import drive

drive.mount('/content/drive')
data_path = '/content/drive/My Drive/Colab Notebooks/NextLab/news_class9x1400'

Mounted at /content/drive


In [7]:
def text_splicing(text, max_len) :
  before_idx = 0
  li = []
  i = 0
  while True:
    if before_idx+max_len < len(text) :
      li.append( [text[before_idx:before_idx+max_len], i] )
      i += 1
    else :
      li.append(  [text[before_idx:], i] )
      if len(li) < 2 :
        return None   # 연결할 단락이 없으므로 dataset에 넣지 않는다.
      return li

    before_idx += max_len
  return li

In [8]:
# 두 문장을 이어붙일 것이기 때문에 256/2인 128로 text를 나눴다.
max_len = 128

In [9]:
text = """특정한 풍경을 통하여 개인적인 감정을 시각화한 사진전
박진호는 1990년대에 자신의 벗은 몸을 복사기로 복제하여 실험적인 결과물을 생산한 작가로서 유명하다.
그런데 이번에는 새벽녘의 달과 하늘을 카메라 앵글에 담아서 감상적인 이미지를 전시하였다.
작가가 이번에 인사동에 있는 나우 갤러리에서 전시한 작품들은 달빛과 구름 낀 새벽하늘을 감성적인 느낌이 드는 결과물로 재구성하여 보여주고 있다.
전시 작품마다 외형적으로 컬러가 자극적이고 전체적으로 톤도 어두워서 보는 이들을 감성적으로 동화시키는 표현전략을 구사하고 있다는 것을 알 수 있다.
작가가 관심을 갖고 카메라 앵글에 담은 대상을 구체적으로 살펴보면 구름과 산봉우리 그리고 강변 풍경이다.
그런데 왠지 표현대상과 소재가 낯설게 느껴지지 않고 너무나도 익숙하다.
왜 그런 것 일까?
그것은 작가가 표현대상으로 선택한 소재와 표현방식이 ‘일요 사진가’라고 일컬어지는 아마추어 작가들이 흔히 찍는 탐미적인 사진과 별다른 차이점이 없기 때문이다.
사진은 시각예술이다.
그러므로 작가가 표현하고자하는 주제와 관계없이 외형적으로 보여 지고 느껴지는 것이 무엇보다도 중요하다.
그런데도 아쉽게도 이번에 박진호가 발표한 풍경사진들은 외형적으로 아마추어 작가들의 유미주의적이고 감상적인 작품과의 차별화에 실패하였다.
풍경사진이나 정물사진은 절제된 프레이밍과 세련된 컬러와 톤이 작품의 완성도를 보장하는데 있어서 중요한 요소로 작용한다.
그러므로 주제선택과 더불어서 외형적으로는 그것을 좀 더 염두에 두고 작업을 진행을 했다면 최종 결과물의 완성도가 달라졌을 것이다.
이번에 작가가 발표한 작품들은 누구나 폭넓게 이해할 수 있는 소재와 표현방식을 보여주고 있다. 하지만 자신만의 조형언어를 보여주는 데는 소홀히 했다는 것이 느껴진다.
그래서 여러 가지로 아쉬운 점이 많은 전시가 되었다. 하지만 자신의 감정을 솔직하게 표현하여 결과물을 생산 한 것은 분명하다.
특정한 풍경을 통하여 개인적인 감정을 시각화 전시이다.""".replace('\n', '')

text_splicing(text, max_len) 

[['특정한 풍경을 통하여 개인적인 감정을 시각화한 사진전박진호는 1990년대에 자신의 벗은 몸을 복사기로 복제하여 실험적인 결과물을 생산한 작가로서 유명하다.그런데 이번에는 새벽녘의 달과 하늘을 카메라 앵글에 담아서 감상적인 이미지를',
  0],
 [' 전시하였다.작가가 이번에 인사동에 있는 나우 갤러리에서 전시한 작품들은 달빛과 구름 낀 새벽하늘을 감성적인 느낌이 드는 결과물로 재구성하여 보여주고 있다.전시 작품마다 외형적으로 컬러가 자극적이고 전체적으로 톤도 어두워서 보는 ',
  1],
 ['이들을 감성적으로 동화시키는 표현전략을 구사하고 있다는 것을 알 수 있다.작가가 관심을 갖고 카메라 앵글에 담은 대상을 구체적으로 살펴보면 구름과 산봉우리 그리고 강변 풍경이다.그런데 왠지 표현대상과 소재가 낯설게 느껴지지 않고 ',
  2],
 ['너무나도 익숙하다.왜 그런 것 일까?그것은 작가가 표현대상으로 선택한 소재와 표현방식이 ‘일요 사진가’라고 일컬어지는 아마추어 작가들이 흔히 찍는 탐미적인 사진과 별다른 차이점이 없기 때문이다.사진은 시각예술이다.그러므로 작가가 ',
  3],
 ['표현하고자하는 주제와 관계없이 외형적으로 보여 지고 느껴지는 것이 무엇보다도 중요하다.그런데도 아쉽게도 이번에 박진호가 발표한 풍경사진들은 외형적으로 아마추어 작가들의 유미주의적이고 감상적인 작품과의 차별화에 실패하였다.풍경사진이',
  4],
 ['나 정물사진은 절제된 프레이밍과 세련된 컬러와 톤이 작품의 완성도를 보장하는데 있어서 중요한 요소로 작용한다.그러므로 주제선택과 더불어서 외형적으로는 그것을 좀 더 염두에 두고 작업을 진행을 했다면 최종 결과물의 완성도가 달라졌을',
  5],
 [' 것이다.이번에 작가가 발표한 작품들은 누구나 폭넓게 이해할 수 있는 소재와 표현방식을 보여주고 있다. 하지만 자신만의 조형언어를 보여주는 데는 소홀히 했다는 것이 느껴진다.그래서 여러 가지로 아쉬운 점이 많은 전시가 되었다. 하',
  6],
 ['지만 자신의 감정을 솔직하게 표

In [10]:
import os
content = []
for (path, dir, files) in os.walk(data_path):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.txt':
            with open("%s/%s" % (path, filename), encoding="utf-8") as f:
              label = path[path.rindex('/')+1:]
              text = f.read()
              temp_list = text_splicing(text.replace('\n', ' '), max_len)
              if temp_list is not None :
                content.extend(temp_list)

len(content), len(content[0]) # content : [(text, 단락 번호)]

(240216, 2)

In [11]:
len(content)

240216

In [12]:
# label : 0 is True, 1 is False
import random
i, len_content = 0, len(content)
dataset = []
right_li, wrong_li = [], []   # 추가할 list
before_text = content[0][0]
while i < len_content :
  text = content[i][0]
  idx = content[i][1]
  if idx > 0 :    # label : right
    right_li.append( (before_text, text, 0) )   # right

  # idx == 0 and len(li) == 1 인 경우는 없다. file read할 때 넣지 않았다. 
  elif idx == 0 and len(right_li) > 1 :   # label : wrong 
    len_li = len(right_li)
    for t in right_li :
      rd = random.randint(0, len_content-len_li-1)
      wrong_text = content[rd][0] if rd < i-len_li else content[rd+len_li][0]   # random으로 고르기 ( 같은 기사 제외 )
      wrong_li.append((t[0], wrong_text, 1) )
    dataset.extend(right_li)
    dataset.extend(wrong_li)
    right_li, wrong_li = [], []
  before_text = text
  i += 1
len_content, len(dataset)

(240216, 455188)

In [13]:
dataset[-3:]

[(' 의미에서 종택의 이름도 그렇게 정했다고 한다. 현판은 안동 퇴계종손이 직접 써준 것이라고 한다. 무송헌(撫松軒) 김담(金淡, 1416년~1464년)선생은 영주출신으로 호는 무송헌이며 조선 세종~문종 때의 명신이다. 이조 판서를 ',
  '. 과잉진료와 오진으로 인한 피해가 조기 검진의 효과보다 크고, 유방암 사망률의 감소는 대부분 치료 기술이 좋아졌기 때문이다. 일부 의사들과 미국암학회는 강력하게 반발하며 현행대로 40세부터 매해 유방암 검사를 받아야 한다고 주장',
  1),
 ('지냈다. 시호는 문절(文節)공이며, 단계서원(丹溪書院), 구강서원(龜江書院)에 배향되어 있다. 그는 세계적인 수준의 역법서를 제작한 천문학자로 세종대왕 당시 집현전 학사로 17년간 재직했다. 재직 시 칠정산(七政算) 내외편(內外篇',
  ' 했다. 경남민언련은 22일 경남도청에서 기자회견을 열어 공동 대응 방침을 밝혔다. 경남민언련은 홍준표 지사의 기자를 상대로 한 소송을 접하면서 먼저, 언론의 감시와 견제의 대상이 되는 선출직 공무원의 소송 남발이 매우 우려되고 ',
  1),
 (')의 편찬을 주도했다. 칠정산 내외편은 세종24년(1442)에 완성되었는데, 전문가들은 아랍의 천문학보다 앞선 동 시대 세계에서 가장 앞선 천문 계산술로 평가하고 있다. 일본이 이러한 수준의 연구를 한 것이 1682년이라고 하니 ',
  '다. 하지만 지자체의 사업조정제도 관련 준비태세 부족과 상인들의 요구가 맞물려 지자체는 벌써부터 홍역을 치르고 있다. 지자체도 중기청에 대해 불만이 있으나 말을 못하고 있을 뿐이다. 대형마트규제인천대책위는 사업조정 관련 절차 세칙',
  1)]

In [14]:
from sklearn.model_selection import train_test_split

# Split Train and Validation data
train, test = train_test_split(dataset, test_size=0.2, random_state=0, shuffle=True)
train, val = train_test_split(train, test_size=0.2, random_state=0, shuffle=True)
print("train :", len(train), "  val :", len(val), "  test :", len(test))

train : 291320   val : 72830   test : 91038


In [15]:
del content
del dataset
del right_li
del wrong_li
del before_text
del text

In [16]:
train[0]

('무슨 책을 읽어야 할지 모른겠다면 이 책을 보라 가끔 사람들이 내게 묻는 말이 있다. 살아가면서 즐거운 일이 있냐고? 그때 난 말한다. 즐거운 일이 뭐가 있겠는가. 고달프고 부대끼면서 사는 거지. 그러면서 한 가지 덧붙이는 게 있',
 ' 휴업기간은 4월 3일부터 5월 31일까지다. 폐업 방침을 발표했던 지난 2월 26일 당시 진주의료원 입원 환자는 200명이 넘었는데, 현재 3명만 남아 있다. 홍 지사가 이번 주 안에 진주의료원 폐업 발표를 할 것으로 보인다. ',
 1)

# 모델 선언

## CorpusDataset

In [17]:
class CorpusDataset(Dataset):
    def __init__(self, sentences, transform: Callable):
        self.sentences = sentences
        self.transform = transform

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        prompt = self.sentences[idx][0]
        next_sentence = self.sentences[idx][1]
        label = self.sentences[idx][2]
        (
            input_ids,
            attention_mask,
            token_type_ids,
            label, 
        ) = self.transform(prompt, next_sentence, label)

        return input_ids, attention_mask, token_type_ids, label

## Preprocessor

In [18]:
from typing import List, Tuple

class Preprocessor :
    def __init__(self, max_len: int):
        self.tokenizer = BertTokenizer.from_pretrained("snunlp/KR-Medium", do_lower_case=False)
        self.max_len = max_len
        self.pad_token_id = 0

    def get_input_features(self, prompt, next_sentence, label
    ) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
        """두 문장(prompt, next_sentence)에 대해 tokenize한 뒤 id로 변환, 두 문장을 이버붙인다.

        Args:
            prompt: 이전 문장
            next_sentence: 이어질 다음 문장
            label : 두 문장이 이어지면 0, 아니면 1

        Returns:
            feature를 리턴한다.
            input_ids : 각 토큰의 id, 2(CLS)로 시작, 3(SEP)
            attention_mask : padding은 0, 데이터가 존재하면 1
            token_type_ids : prompt 위치는 0, next_sentence 위치는 1, 1 뒤의 0은 패딩
            label : 정답이면 [1, 0], 오답이면 [0, 1]
        """
        tok_prompt = self.tokenizer.tokenize(prompt)
        tok_next_sen = self.tokenizer.tokenize(next_sentence)

        input_ids = [2] + self.tokenizer.convert_tokens_to_ids(tok_prompt) + [3]    # 처음과 끝 표시
        input_ids_next_sen = [2]+ self.tokenizer.convert_tokens_to_ids(tok_next_sen)+ [3]

        slicing_idx = 0
        if len(input_ids) + len(input_ids_next_sen) > self.max_len :
          slicing_idx = len(input_ids) + len(input_ids_next_sen) - self.max_len // 2 + 2
          input_ids = [2] + input_ids[slicing_idx:]
          input_ids_next_sen = input_ids_next_sen[:-slicing_idx] + [3]

        token_type_ids = [0]*len(input_ids)
        token_type_ids.extend([1]*len(input_ids_next_sen))
        input_ids.extend(input_ids_next_sen)
        attention_mask = [1] * len(token_type_ids)
        pad_length = self.max_len-len(input_ids)

        input_ids.extend([0] * pad_length)
        token_type_ids.extend([0] * pad_length) # pad : 0
        attention_mask.extend([0] * pad_length) # pad : 0

        input_ids = torch.tensor(input_ids, dtype=torch.int)
        attention_mask = torch.tensor(attention_mask, dtype=torch.int)
        token_type_ids = torch.tensor(token_type_ids, dtype=torch.int)

        label = [1.0, 0.] if label == 0 else [0., 1.0]
        label = torch.tensor(label, dtype=torch.float)
        return input_ids, attention_mask, token_type_ids, label

## BERTNextSentenceModel

In [19]:
from math import log
class BertOnlyNSPHead(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        self.seq_relationship = nn.Linear(config.hidden_size, 2)

    def forward(self, pooled_output):
        seq_relationship_score = self.seq_relationship(pooled_output)
        print(seq_relationship_score, len(seq_relationship_score))
        return seq_relationship_score

# NextSentencePredictorOutput
class BERTNextSentenceModel(pl.LightningModule):
    def __init__(self, config, dataset):
        super().__init__()
        self.config = config
        self.dataset = dataset
        self.labels_type = [0,1]
        self.pad_token_id = 0
        self.softmax = torch.nn.Softmax()
        self.bert_config = BertConfig.from_pretrained(
            self.config.bert_model, num_labels=2
        )
        self.model = BertForNextSentencePrediction.from_pretrained(
            self.config.bert_model, config=self.bert_config
        )
        self.cls = BertOnlyNSPHead(config)
        self.dropout = nn.Dropout(self.config.dropout_rate)
        self.linear = nn.Linear(
            self.bert_config.hidden_size, len(self.labels_type)
        )

    def forward(self,
        input_ids=None, attention_mask=None, token_type_ids=None, position_ids=None,
        head_mask=None, inputs_embeds=None, labels=None, output_attentions=None, output_hidden_states=None,
        return_dict=None, **kwargs, ):
        """
          self.model의 결과물 :  
          
          NextSentencePredictorOutput(
            loss=next_sentence_loss,
            logits=seq_relationship_scores,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
          )

        """
        logits = self.model(input_ids, token_type_ids=token_type_ids).logits
        probs = self.softmax(logits)
        return probs

    def training_step(self, batch, batch_nb):
        input_ids, attention_mask, token_type_ids, label_ids = batch
        outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            labels=label_ids,
        )
        loss = self._calculate_loss(outputs, label_ids) # slot_labels : labels
        acc = self._calculate_accuracy(outputs, label_ids)
        tensorboard_logs = {'train_loss': loss, 'train_acc':acc}
        return {"loss": loss, "acc": acc, "log": tensorboard_logs}

    def training_end(self, batch, batch_nb):
        input_ids, attention_mask, token_type_ids, label_ids = batch

        outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        loss = self._calculate_loss(outputs, label_ids) # slot_labels : labels
        acc = self._calculate_accuracy(outputs, label_ids)

        tensorboard_logs = {'train_loss': loss, 'train_acc':acc}
        print("training_end : ", tensorboard_logs)
        return {"loss": loss, "acc": acc, "log": tensorboard_logs}

    def training_epoch_end(self, outputs):
        super().on_train_epoch_end()
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        acc = torch.stack([x['acc'] for x in outputs]).mean()
        tensorboard_logs = {'loss': avg_loss, 'acc': acc}
        print('training_epoch_end : ', tensorboard_logs)
        self.log("training_epoch_end : tensorboard_logs ", tensorboard_logs)

    def validation_step(self, batch, batch_nb):
        input_ids, attention_mask, token_type_ids, slot_label_ids = batch
        outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        loss = self._calculate_loss(outputs, slot_label_ids)
        val_acc = self._calculate_accuracy(outputs, slot_label_ids)
        return {"val_loss": loss, 'val_acc':val_acc}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        val_acc = torch.stack([x['val_acc'] for x in outputs]).mean()
        tensorboard_logs = {'val_loss': avg_loss, 'val_acc':val_acc}
        print('validation_epoch_end : ', tensorboard_logs)
        self.log('val_acc', val_acc)
        self.log('val_loss', avg_loss)
        return {'val_acc':val_acc, 'val_loss': avg_loss, 'log': tensorboard_logs}
    
    def validation_end(self, outputs):
        val_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        val_acc = torch.stack([x["val_acc"] for x in outputs]).mean()
        tensorboard_logs = {"val_loss": val_loss, "val_acc" : val_acc }
        return {'val_acc':val_acc, 'val_loss': val_loss, 'log': tensorboard_logs}
    
    def test_step(self, batch, batch_nb):
        input_ids, attention_mask, token_type_ids, slot_label_ids = batch
        outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        acc = self._calculate_accuracy(outputs, slot_label_ids)
        loss = self._calculate_loss(outputs, slot_label_ids)
        return {"test_loss": loss, "test_acc": acc}

    def test_end(self, outputs):
        test_loss = torch.stack([x["test_loss"] for x in outputs]).mean()
        test_acc = torch.stack([x["test_acc"] for x in outputs]).mean()
        self.log("test_loss", test_loss)
        self.log("test_acc", test_acc)
        return {"labels" : [x["labels"] for x in outputs], "test_loss": test_loss,  "test_acc": test_acc}
    
    def test_epoch_end(self, outputs):
        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        acc = torch.stack([x['test_acc'] for x in outputs]).mean()
        tensorboard_logs = {'test_loss': avg_loss, 'test_acc':acc}
        print('test_epoch_end : ', tensorboard_logs)
        self.log("test_epoch_end : tensorboard_logs ", tensorboard_logs)
        return {'test_acc':acc, 'test_loss': avg_loss, 'log': tensorboard_logs}

    def predict_step(self, batch, batch_idx, dataloader_idx=0):   # prediction : forward(), predict_step()
        input_ids, attention_mask, token_type_ids, _ = batch    # slot_label은 없음.
        outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )

        return {'pred_labels':outputs}

    def configure_optimizers(self):
        return AdamW(self.model.parameters(), lr=2e-5, eps=1e-8)

    def train_dataloader(self):
        return DataLoader(self.dataset["train"], batch_size=self.config.eval_batch_size)

    def val_dataloader(self):
        return DataLoader(self.dataset["val"], batch_size=self.config.eval_batch_size)

    def test_dataloader(self):
        return DataLoader(self.dataset["test"], batch_size=self.config.eval_batch_size)

    def pred_dataloader(self, dataset):
        return DataLoader(dataset, batch_size=1)

    def _calculate_loss(self, outputs, labels):   # 확률에서 얼마나 떨어져있는가?
        loss = F.cross_entropy(outputs, labels)
        return loss
        
    def _calculate_accuracy(self, outputs, labels):   # 0.5보다 크면 1, 아니면 0으로 labeling
        active_logits = torch.argmax(outputs, dim=1)  # 큰 값의 index(위치) 가져오기 ( 128 batch이기 때문에 )
        active_labels = torch.argmax(labels, dim=1)
        accuracy = accuracy_score(active_logits, active_labels)
        return accuracy

## Config

In [ ]:
# yaml 파일 대신에 객체로 생성
class Config(BertConfig):
  def __init__(self) :
    super().__init__()
    self.task= 'kor_nextsentence_prediction_'
    self.log_path= data_path+'/logs'
    self.bert_model = "snunlp/KR-Medium"
    self.max_len= 256
    self.train_batch_size= 32
    self.eval_batch_size= 32
    self.dropout_rate= 0.1
    self.gpus= torch.cuda.device_count()
config = Config()

# Train

loss는 softmax한 값에서 label( [0, 1] 혹은 [1, 0] )과 얼마나 떨어져있는가?

acc는 softmax(확률) 값에서 더 큰 index를 label로 지정, output : [0, 1] 혹은 [1, 0], 따라서 성능측정은 acc를 보는 것이 더 낫다.

In [ ]:
preprocessor = Preprocessor(config.max_len)

In [ ]:
dataset = {}
dataset['train'] = CorpusDataset(train, preprocessor.get_input_features)
dataset['val'] = CorpusDataset(val, preprocessor.get_input_features)
dataset['test'] = CorpusDataset(test, preprocessor.get_input_features)

In [ ]:
import gc
gc.collect()

101

In [ ]:
model = BERTNextSentenceModel(config, dataset).cuda()

init


Some weights of the model checkpoint at snunlp/KR-Medium were not used when initializing BertForNextSentencePrediction: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.callbacks import LearningRateMonitor

logger = TensorBoardLogger(
    save_dir=os.path.join(config.log_path, config.task), version=3, name=config.task
)

acc_checkpoint_callback = ModelCheckpoint(
    dirpath=data_path+'/sts/checkpoints/'+ config.task, 
    filename="{epoch}_{val_acc:2f}_{other_metric:.2f}",
    verbose=True,
    monitor='val_acc',
    mode='max',
    save_top_k=1,
    save_last=True)

lrmonitor_callback = LearningRateMonitor(logging_interval='step')

In [ ]:
import gc
gc.collect()

0

In [ ]:
trainer = pl.Trainer(
    gpus=config.gpus,
    callbacks=[acc_checkpoint_callback, lrmonitor_callback],
    logger=logger,
    max_epochs=1,
)

trainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,

  | Name    | Type                          | Params
----------------------------------------------------------
0 | softmax | Softmax                       | 0     
1 | model   | BertForNextSentencePrediction | 101 M 
2 | cls     | BertOnlyNSPHead               | 1.5 K 
3 | dropout | Dropout                       | 0     
4 | linear  | Linear                        | 1.5 K 
----------------------------------------------------------
101 M     Trainable params
0         Non-trainable params
101 M     Total params
405.624   Total estimat

Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


validation_epoch_end :  {'val_loss': tensor(0.3133, device='cuda:0'), 'val_acc': tensor(1., device='cuda:0')}


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/optimization/closure.py:36: LightningDeprecationWarning: One of the returned values {'acc', 'log'} has a `grad_fn`. We will detach it automatically but this behaviour will change in v1.6. Please detach it manually: `return {'loss': ..., 'something': something.detach()}`
  f"One of the returned values {set(extra.keys())} has a `grad_fn`. We will detach it automatically"


Validating: 0it [00:00, ?it/s]

validation_epoch_end :  {'val_loss': tensor(0.3333, device='cuda:0'), 'val_acc': tensor(0.9793, device='cuda:0')}
training_epoch_end
training_epoch_end :  {'loss': tensor(0.3282, device='cuda:0'), 'acc': tensor(0.9845, device='cuda:0')}


Epoch 0, global step 9103: val_acc reached 0.97934 (best 0.97934), saving model to "/content/drive/My Drive/Colab Notebooks/NextLab/news_class9x1400/sts/checkpoints/kor_nextsentence_prediction_/epoch=0_val_acc=0.979335_other_metric=0.00.ckpt" as top 1
Saving latest checkpoint...


In [ ]:
# 2epoch
trainer.test()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1399: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `test(ckpt_path='best')` to use and best model checkpoint and avoid this warning or `ckpt_path=trainer.checkpoint_callback.last_model_path` to use the last model.
  f"`.{fn}(ckpt_path=None)` was called without a model."
Restoring states from the checkpoint path at /content/drive/My Drive/Colab Notebooks/NextLab/news_class9x1400/sts/checkpoints/kor_nextsentence_prediction_/acc_epoch=1_val_loss=                           0.331538.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at /content/drive/My Drive/Colab Notebooks/NextLab/news_class9x1400/sts/checkpoints/kor_nextsentence_prediction_/acc_epoch=1_val_loss=                           0.331538.ckpt
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWar

Testing: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


test_epoch_end :  {'test_loss': tensor(0.3316, device='cuda:0'), 'test_acc': tensor(0.9807, device='cuda:0')}
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_epoch_end : tensorboard_logs ': {'test_acc': tensor(0.9807, device='cuda:0'),
                                        'test_loss': tensor(0.3316, device='cuda:0')}}
--------------------------------------------------------------------------------


[{'test_epoch_end : tensorboard_logs ': {'test_acc': tensor(0.9807, device='cuda:0'),
   'test_loss': tensor(0.3316, device='cuda:0')}}]

In [ ]:
# 3 epoch
trainer.test()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1399: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `test(ckpt_path='best')` to use and best model checkpoint and avoid this warning or `ckpt_path=trainer.checkpoint_callback.last_model_path` to use the last model.
  f"`.{fn}(ckpt_path=None)` was called without a model."
Restoring states from the checkpoint path at /content/drive/My Drive/Colab Notebooks/NextLab/news_class9x1400/sts/checkpoints/kor_nextsentence_prediction_/epoch=0_val_acc=0.979335_other_metric=0.00.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at /content/drive/My Drive/Colab Notebooks/NextLab/news_class9x1400/sts/checkpoints/kor_nextsentence_prediction_/epoch=0_val_acc=0.979335_other_metric=0.00.ckpt
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_d

Testing: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


test_epoch_end :  {'test_loss': tensor(0.3329, device='cuda:0'), 'test_acc': tensor(0.9796, device='cuda:0')}
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_epoch_end : tensorboard_logs ': {'test_acc': tensor(0.9796, device='cuda:0'),
                                        'test_loss': tensor(0.3329, device='cuda:0')}}
--------------------------------------------------------------------------------


[{'test_epoch_end : tensorboard_logs ': {'test_acc': tensor(0.9796, device='cuda:0'),
   'test_loss': tensor(0.3329, device='cuda:0')}}]

In [ ]:
trainer.save_checkpoint(data_path+'/sts/checkpoints_acc_9793.ckpt')

# 다른 모델로 학습

In [20]:
class Config(BertConfig):
  def __init__(self) :
    super().__init__()
    self.task= 'kor_nextsentence_prediction_klue'
    self.log_path= data_path+'/logs'
    self.bert_model = 'klue/bert-base'
    self.max_len= 256
    self.train_batch_size= 32
    self.eval_batch_size= 32
    self.dropout_rate= 0.1
    self.gpus= torch.cuda.device_count()
config = Config()

In [21]:
preprocessor = Preprocessor(config.max_len)
dataset = {}
dataset['train'] = CorpusDataset(train, preprocessor.get_input_features)
dataset['val'] = CorpusDataset(val, preprocessor.get_input_features)
dataset['test'] = CorpusDataset(test, preprocessor.get_input_features)

Downloading:   0%|          | 0.00/140k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/337 [00:00<?, ?B/s]

In [22]:
model = BERTNextSentenceModel(config, dataset)

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForNextSentencePrediction: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [23]:
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.callbacks import LearningRateMonitor

logger = TensorBoardLogger(
    save_dir=os.path.join(config.log_path, config.task), version=3, name=config.task
)

acc_checkpoint_callback = ModelCheckpoint(
    dirpath=data_path+'/sts/checkpoints/'+ config.task, 
    filename="{epoch}_{val_acc:2f}_{other_metric:.2f}",
    verbose=True,
    monitor='val_acc',
    mode='max',
    save_top_k=1,
    save_last=True)

lrmonitor_callback = LearningRateMonitor(logging_interval='step')

In [24]:
import gc
gc.collect()

527

In [25]:
trainer = pl.Trainer(
    gpus=config.gpus,
    callbacks=[acc_checkpoint_callback, lrmonitor_callback],
    logger=logger,
    max_epochs=3,
)

trainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,

  | Name    | Type                          | Params
----------------------------------------------------------
0 | softmax | Softmax                       | 0     
1 | model   | BertForNextSentencePrediction | 110 M 
2 | cls     | BertOnlyNSPHead               | 1.5 K 
3 | dropout | Dropout                       | 0     
4 | linear  | Linear                        | 1.5 K 
----------------------------------------------------------
110 M     Trainable params
0         Non-trainable params
110 M     Total params
442.488   Total estimat

Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


validation_epoch_end :  {'val_loss': tensor(0.7965, device='cuda:0'), 'val_acc': tensor(0.5156, device='cuda:0')}


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


...

Epoch 1, global step 18207: val_acc reached 0.92069 (best 0.92069), saving model to "/content/drive/My Drive/Colab Notebooks/NextLab/news_class9x1400/sts/checkpoints/kor_nextsentence_prediction_klue/epoch=1_val_acc=0.920692_other_metric=0.00.ckpt" as top 1


{'val_loss': tensor(0.3900, device='cuda:0'), 'val_acc': tensor(0.9207, device='cuda:0')}
training_epoch_end :  {'loss': tensor(0.3896, device='cuda:0'), 'acc': tensor(0.9210, device='cuda:0')}


Validating: 0it [00:00, ?it/s]

validation_epoch_end :  {'val_loss': tensor(0.3855, device='cuda:0'), 'val_acc': tensor(0.9246, device='cuda:0')}
training_epoch_end :  {'loss': tensor(0.3796, device='cuda:0'), 'acc': tensor(0.9315, device='cuda:0')}


Epoch 2, global step 27311: val_acc reached 0.92462 (best 0.92462), saving model to "/content/drive/My Drive/Colab Notebooks/NextLab/news_class9x1400/sts/checkpoints/kor_nextsentence_prediction_klue/epoch=2_val_acc=0.924620_other_metric=0.00.ckpt" as top 1
Saving latest checkpoint...


In [26]:
trainer.test()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1399: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `test(ckpt_path='best')` to use and best model checkpoint and avoid this warning or `ckpt_path=trainer.checkpoint_callback.last_model_path` to use the last model.
  f"`.{fn}(ckpt_path=None)` was called without a model."
Restoring states from the checkpoint path at /content/drive/My Drive/Colab Notebooks/NextLab/news_class9x1400/sts/checkpoints/kor_nextsentence_prediction_klue/epoch=2_val_acc=0.924620_other_metric=0.00.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at /content/drive/My Drive/Colab Notebooks/NextLab/news_class9x1400/sts/checkpoints/kor_nextsentence_prediction_klue/epoch=2_val_acc=0.924620_other_metric=0.00.ckpt
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader

Testing: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


test_epoch_end :  {'test_loss': tensor(0.3870, device='cuda:0'), 'test_acc': tensor(0.9230, device='cuda:0')}
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_epoch_end : tensorboard_logs ': {'test_acc': tensor(0.9230, device='cuda:0'),
                                        'test_loss': tensor(0.3870, device='cuda:0')}}
--------------------------------------------------------------------------------


[{'test_epoch_end : tensorboard_logs ': {'test_acc': tensor(0.9230, device='cuda:0'),
   'test_loss': tensor(0.3870, device='cuda:0')}}]